<a href="https://colab.research.google.com/github/rodimendes/rest-analysis/blob/main/vendas2018_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importação de bibliotecas e pastas

In [1]:
import pandas as pd
import os
import pathlib
import sqlite3
import datetime as dt

In [2]:
from google.colab import drive
drive.mount('/ShardDrives/')

Mounted at /ShardDrives/


In [3]:
pd.set_option('float_format', '{:,.2f}'.format)

## Renomeando arquivos 
**Desnecessário para estes arquivos** 

Executado e nomes alterados

In [4]:
"""folder = "/ShardDrives/MyDrive/rest_data_extraction /volume_vendas_produto/"
for file in os.listdir(folder):
    old_name = folder + file
    data = pd.read_excel(old_name)
    if file[:7] != 'vendas_':
      report_date = data['Unnamed: 1'][1][-10:]
      new_name = f"{folder}vendas_{report_date}.xlsx"
      os.rename(old_name, new_name)"""

'folder = "/ShardDrives/MyDrive/rest_data_extraction /volume_vendas_produto/"\nfor file in os.listdir(folder):\n    old_name = folder + file\n    data = pd.read_excel(old_name)\n    if file[:7] != \'vendas_\':\n      report_date = data[\'Unnamed: 1\'][1][-10:]\n      new_name = f"{folder}vendas_{report_date}.xlsx"\n      os.rename(old_name, new_name)'

## Data opening

In [5]:
file = "/ShardDrives/MyDrive/rest_data_extraction /base_de_dados/volume_vendas_2018_2022.xlsx"
data = pd.read_excel(file)

FileNotFoundError: ignored

## Visão geral do dataframe

In [ ]:
data.head(10)

## Limpeza inicial

In [ ]:
# Limpar linhas vazias e reiniciar o index

df = data.rename(columns=data.iloc[7]).iloc[8:-1].reset_index(drop=True)
df

In [ ]:
# Separando coluna Famila / Sub-Familia em duas

new_column = df['Familia / Sub-Familia'].str.split('/', n=1, expand=True)
df['familia'] = new_column[0]
df['sub_familia'] = new_column[1]

In [ ]:
# Renomeando colunas

df = df[['Data', 'Código', 'Produto', 'familia', 'sub_familia', 'Quantidade', 'Valor Total S/IVA', 'Valor Total']]
df.columns = ['data', 'codigo', 'produto', 'familia', 'sub_familia', 'quantidade', 'total_s_iva', 'total']

### Tratamento de colunas


In [ ]:
# Definindo o tipo adequado para as colunas

df.info()

In [ ]:
df.data = pd.to_datetime(df.data)
df['codigo'] = pd.to_numeric(df['codigo'])
df['quantidade'] = pd.to_numeric(df['quantidade'])
df['total_s_iva'] = pd.to_numeric(df['total_s_iva'])
df['total'] = pd.to_numeric(df['total'])
df.info()

In [ ]:
df.columns

## Criação do Banco de Dados
**Etapa desnecessária para este caso de já termos o dataset completo e não ser necessário atualizar com novos dados**


In [ ]:
#bd = sqlite3.connect("vendas_restaurantes.db")

In [ ]:
#cursor = bd.cursor()

### A célula abaixo só é necessário executar na primeira vez, para a criação da tabela no banco de dados

In [ ]:
#cursor.execute("CREATE TABLE 'Vendas Restauração 2018_2022' (data INTEGER, codigo INT, produto TEXT, familia TEXT, sub_familia TEXT, quantidade INT, total_s_iva REAL, total REAL)")

In [ ]:
#df.head(2)

In [ ]:
#for index, row in df.iterrows():
#    data = row[0]
#    codigo = row[1]
#    produto = row[2]
#    familia = row[3]
#    sub_familia = row[4]
#    quantidade = row[5]
#    total_s_iva = row[6]
#    total = row[7]
#    cursor.execute('''INSERT INTO 'Vendas Restauração 2018_2022' \
#                    (data, codigo, produto, familia, sub_familia, quantidade, total_s_iva, total) VALUES (?,?,?,?,?,?,?,?)''', \
#                    (str(data), codigo, produto, familia, sub_familia, quantidade, total_s_iva, total))
#    
#    bd.commit()

##**Primeiras explorações**

In [ ]:
# Faturamento total do período

faturamento_total = df.total.sum()
f'A empresa faturou {faturamento_total:,.2f} de euros durante o período'

In [ ]:
# Faturamento por família

faturamento_familia = df[['familia', 'total']].groupby(by='familia').sum()
faturamento_familia.sort_values(by='total', ascending=False)

###Rentabilidade diária

In [ ]:
dt.date.isoweekday(df.data[0])

In [ ]:
# Criando uma lista com todos os dias de venda e atribuindo a uma nova coluna

dia_semana = []
for dia in df.data:
  dia_semana.append(dt.date.isoweekday(dia))

df['dia_semana'] = dia_semana
df.head(2)

In [ ]:
rentabilidade_diaria = df[['quantidade','total', 'dia_semana']].groupby(by='dia_semana').sum()
rentabilidade_diaria

In [ ]:
# Rentabilidade diária, em ordem decrescente, para o período.

rentabilidade_diaria.sort_values(by='total', ascending=False)

In [ ]:
# Dia da semana possui mais entradas, ou seja, mais linhas.

df.dia_semana.value_counts()

In [ ]:
## Por que as segundas e possuem valores muito baixos?
## Quais os dias e horários de funcionamento do restaurante?

In [ ]:
df[df['dia_semana'] == 1]

##**Regressão Linear - Faturamento futuro**##

In [ ]:
# Bibliotecas para construção do modelo

import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

In [ ]:
object_df = df.select_dtypes(include=['object']).copy()

In [ ]:
object_df

In [ ]:
anos = []
for dia in df['data']:
  year = dia.year
  anos.append(year)

print(len(anos))

In [ ]:
df['ano'] = anos

In [ ]:
# Agrupar por anos e analisar ano a ano

In [ ]:
# Célula teste

df.sample()

In [ ]:
df_2018 = df[df['ano'] == 2018]
df_2019 = df[df['ano'] == 2019]
df_2020 = df[df['ano'] == 2020]
df_2021 = df[df['ano'] == 2021]
df_2022 = df[df['ano'] == 2022]

In [ ]:
print(f'Ano 2018 - {df_2018.shape}')
print(f'Ano 2019 - {df_2019.shape}')
print(f'Ano 2020 - {df_2020.shape}')
print(f'Ano 2021 - {df_2021.shape}')
print(f'Ano 2022 - {df_2022.shape}')

In [ ]:
df[['quantidade', 'total', 'ano']].groupby(by='ano').sum()

In [ ]:
# Fazer o git funcionar para o trabalho
"""
Para abrir:
  Clicar ARQUIVO >> ABRIR NOTEBOOK
  Selecionar a aba GITHUB e localizar o repositório 'rodimendes/rest-analysis', o branch MAIN e clicar no arquivo 'vendas2018_2022.ipynb'
Ao sair:
  Clicar ARQUIVO >> SALVAR UMA CÓPIA NO GITHUB
  Na janela que se abrirá, conferir repositório, branch e clicar em OK
"""

In [ ]:
# Agrupar por mês e analisar mês a mês (??)
# Agrupar por dia e analisar o dia a dia
# Fazer RL e identificar o comportamento da sazonalidade e a "outra"
# Fazer Time Series e comparar modelos
# meses = []


In [ ]:
#sandragomes